****OPTIMISATION D'UN SYSTÈME AUTOMASITÉ DE RÉCONCILIATION DE DONNÉES HÉTÉROGÈNES POUR AMÉLIORER LA PRÉCISION DES MODÈLES PRÉDICTIFS ET OPTIMISER LES PERFORMANCES DES TAUX DE MATCHING****

****1. Analyse RFM à l'aide de l'ensemble de données Online Retail****

In [50]:
import numpy as np
import pandas as pd

In [51]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

****1.1. Installer des paquets et définir des configurations****

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

****1.2. Lire les données****

In [3]:
df_ = pd.read_excel("online_retail_II.xlsx") 
df = df_.copy()
print(df)

       Invoice StockCode                          Description  Quantity  \
0       489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1       489434    79323P                   PINK CHERRY LIGHTS        12   
2       489434    79323W                  WHITE CHERRY LIGHTS        12   
3       489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4       489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
...        ...       ...                                  ...       ...   
525456  538171     22271                 FELTCRAFT DOLL ROSIE         2   
525457  538171     22750         FELTCRAFT PRINCESS LOLA DOLL         1   
525458  538171     22751       FELTCRAFT PRINCESS OLIVIA DOLL         1   
525459  538171     20970   PINK FLORAL FELTCRAFT SHOULDER BAG         2   
525460  538171     21931               JUMBO STORAGE BAG SUKI         2   

               InvoiceDate   Price  Customer ID         Country  
0      2009-12-01 07:45:00 6.9500

*****Il y a 525 461 lignes et 8 colonnes dans le dataframe*****

In [5]:
clients_uniques = df[['Customer ID']].drop_duplicates()
print(clients_uniques)

        Customer ID
0       13085.00000
12      13078.00000
31      15362.00000
54      18102.00000
71      12682.00000
...             ...
521212  18269.00000
522042  13270.00000
522621  12942.00000
522900  13369.00000
524732  15211.00000

[4384 rows x 1 columns]


****Il y a 4 384 clients****

In [55]:
df_ = pd.read_excel("online_retail_II.xlsx") 
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


****1.3. Comprendre les données****

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [12]:
df['Country'].nunique()

40

In [13]:
df['Country'].value_counts()

Country
United Kingdom          485852
EIRE                      9670
Germany                   8129
France                    5772
Netherlands               2769
Spain                     1278
Switzerland               1187
Portugal                  1101
Belgium                   1054
Channel Islands            906
Sweden                     902
Italy                      731
Australia                  654
Cyprus                     554
Austria                    537
Greece                     517
United Arab Emirates       432
Denmark                    428
Norway                     369
Finland                    354
Unspecified                310
USA                        244
Japan                      224
Poland                     194
Malta                      172
Lithuania                  154
Singapore                  117
RSA                        111
Bahrain                    107
Canada                      77
Hong Kong                   76
Thailand                    76


In [14]:
df.sort_values(by='Price', ascending=False).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
241824,C512770,M,Manual,-1,2010-06-17 16:52:00,25111.09000,17399.00000,United Kingdom
241827,512771,M,Manual,1,2010-06-17 16:53:00,25111.09000,NaN,United Kingdom
320581,C520667,BANK CHARGES,Bank Charges,-1,2010-08-27 13:42:00,18910.69000,NaN,United Kingdom
517953,C537630,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:04:00,13541.33000,NaN,United Kingdom
519294,C537651,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:49:00,13541.33000,NaN,United Kingdom


In [15]:
df['Description'].nunique()

4681

In [16]:
df.groupby("Description").agg({"Quantity": lambda x: x.sum()}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


In [17]:
print(f"Nombre de pièces uniques dans StockCode: {df['StockCode'].nunique()}")
print(f"Nombre d'exemplaires uniques dans la description:{df['Description'].nunique()}")

Nombre de pièces uniques dans StockCode: 4632
Nombre d'exemplaires uniques dans la description:4681


In [18]:
a =df.groupby('StockCode').agg({'Description': "nunique"})
a.reset_index(inplace=True)
a.head()
b = list(a.loc[a['Description'] > 1, 'StockCode'])   

for urun_kodu in b:
    print(f"urun kodu = {urun_kodu} {df.loc[df['StockCode'] == urun_kodu, 'Description'].unique()}")

urun kodu = 10120 ['DOGGY RUBBER' 'Zebra invcing error']
urun kodu = 16011 [' ANIMAL STICKERS' 'ANIMAL STICKERS']
urun kodu = 16012 ['FOOD/DRINK SPUNGE STICKERS' 'FOOD/DRINK SPONGE STICKERS']
urun kodu = 16235 ['RECYCLED PENCIL WITH RABBIT ERASER' '?']
urun kodu = 17033 ['BROCADE PURSE,SMALL ARCH BUTTON' 'sold as 17003?' 'Sold as 17003?' nan]
urun kodu = 20615 ['BLUE SPOTTY PASSPORT COVER' 'BLUE POLKADOT PASSPORT COVER']
urun kodu = 20652 ['BLUE SPOTTY LUGGAGE TAG ' nan 'BLUE POLKADOT LUGGAGE TAG ']
urun kodu = 20658 ['RED SPOTTY LUGGAGE TAG' 'RED RETROSPOT LUGGAGE TAG']
urun kodu = 20661 ['BLUE SPOTTY PURSE ' 'BLUE POLKADOT PURSE ']
urun kodu = 20665 ['RED SPOTTY PURSE ' 'RED RETROSPOT PURSE ']
urun kodu = 20674 ['GREEN SPOTTY BOWL' 'GREEN POLKADOT BOWL']
urun kodu = 20675 ['BLUE SPOTTY BOWL' 'BLUE POLKADOT BOWL']
urun kodu = 20676 ['RED SPOTTY BOWL' 'RED RETROSPOT BOWL']
urun kodu = 20677 ['PINK SPOTTY BOWL' 'PINK POLKADOT BOWL']
urun kodu = 20679 ['EDWARDIAN PARASOL RED' nan '?']
ur

In [19]:
df.loc[df['StockCode'] == 20677, 'Description'].unique() 

array(['PINK SPOTTY BOWL', 'PINK POLKADOT BOWL'], dtype=object)

****1.4. Préparation des données****

In [21]:
df.dropna(inplace=True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 417534 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  object        
 1   StockCode    417534 non-null  object        
 2   Description  417534 non-null  object        
 3   Quantity     417534 non-null  int64         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      417534 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 28.7+ MB


In [23]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max,std
Quantity,417534.00000,12.75881,-9360.00000,-2.00000,1.00000,1.00000,2.00000,4.00000,12.00000,24.00000,36.00000,144.00000,19152.00000,101.22042
InvoiceDate,417534,2010-07-01 01:12:19.787609856,2009-12-01 07:45:00,2009-12-02 15:04:00,2009-12-13 12:42:00,2010-01-18 14:49:00,2010-03-26 11:26:00,2010-07-08 19:12:00,2010-10-14 14:08:00,2010-11-17 17:16:00,2010-11-28 13:43:00,2010-12-07 14:41:00,2010-12-09 20:01:00,NaN
Price,417534.00000,3.88755,0.00000,0.29000,0.42000,0.65000,1.25000,1.95000,3.75000,6.75000,8.50000,14.95000,25111.09000,71.13180
Customer ID,417534.00000,15360.64548,12346.00000,12435.00000,12725.00000,13042.00000,13983.00000,15311.00000,16799.00000,17706.00000,17913.00000,18196.00000,18287.00000,1680.81132


In [24]:
df = df[~df["Invoice"].str.contains("C", na=False)]

In [25]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max,std
Quantity,407695.00000,13.58669,1.00000,1.00000,1.00000,1.00000,2.00000,5.00000,12.00000,24.00000,36.00000,144.00000,19152.00000,96.84223
InvoiceDate,407695,2010-07-01 10:10:10.782177792,2009-12-01 07:45:00,2009-12-02 15:10:00,2009-12-13 13:27:00,2010-01-19 11:01:00,2010-03-26 14:01:00,2010-07-09 15:46:00,2010-10-14 17:09:00,2010-11-18 10:37:00,2010-11-28 13:53:00,2010-12-07 14:31:00,2010-12-09 20:01:00,NaN
Price,407695.00000,3.29419,0.00000,0.29000,0.42000,0.65000,1.25000,1.95000,3.75000,6.75000,8.50000,14.95000,10953.50000,34.75666
Customer ID,407695.00000,15368.50411,12346.00000,12435.00000,12731.00000,13044.00000,13997.00000,15321.00000,16812.00000,17706.00000,17913.00000,18196.00000,18287.00000,1679.79570


In [26]:
df['TotalPrice'] = df['Quantity'] * df['Price']
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000


****1.5. RFM Metrics****

In [28]:
df['InvoiceDate'].max()

Timestamp('2010-12-09 20:01:00')

In [29]:
today_date = df['InvoiceDate'].max() + dt.timedelta(days=2)
today_date

Timestamp('2010-12-11 20:01:00')

In [30]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                    'Invoice': lambda inv: inv.nunique(),
                                    'TotalPrice': lambda price: price.sum()})
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.00000,166,11,372.86000
12347.00000,4,2,1323.32000
12348.00000,75,1,222.16000
12349.00000,44,3,2671.14000
12351.00000,12,1,300.93000


In [31]:
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346.00000,166,11,372.86000
12347.00000,4,2,1323.32000
12348.00000,75,1,222.16000
12349.00000,44,3,2671.14000
12351.00000,12,1,300.93000


In [32]:
rfm.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Recency,4314.00000,92.26912,96.94348,2.00000,2.00000,4.00000,9.00000,19.00000,54.00000,137.00000,256.00000,305.00000,369.00000,375.00000
Frequency,4314.00000,4.45410,8.16866,1.00000,1.00000,1.00000,1.00000,1.00000,2.00000,5.00000,9.00000,13.00000,31.00000,205.00000
Monetary,4314.00000,2047.28866,8912.52324,0.00000,39.95650,110.43650,155.48500,307.95000,705.55000,1722.80250,3796.59500,6235.12050,20136.78250,349164.35000


****1.6. RFM Scores****

In [34]:
rfm["RecencyScore"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["FrequencyScore"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

In [35]:
rfm["RFM_SCORE"] = (rfm['RecencyScore'].astype(str) +
                    rfm['FrequencyScore'].astype(str) +
                    rfm['MonetaryScore'].astype(str))
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12346.00000,166,11,372.86000,2,5,2,252
12347.00000,4,2,1323.32000,5,2,4,524
12348.00000,75,1,222.16000,2,1,1,211
12349.00000,44,3,2671.14000,3,3,5,335
12351.00000,12,1,300.93000,5,1,2,512


In [36]:
rfm[rfm['RFM_SCORE'] == "555"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12415.00000,12,7,19543.84000,5,5,5,555
12431.00000,10,13,4370.52000,5,5,5,555
12471.00000,11,49,20139.74000,5,5,5,555
12472.00000,6,13,11308.48000,5,5,5,555
12474.00000,15,13,5048.66000,5,5,5,555


****1.7. Naming the RFM Scores****

In [38]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_Loose',
    r'3[1-2]': 'About_to_Sleep',
    r'33': 'Need_Attention',
    r'[3-4][4-5]': 'Loyal_Customers',
    r'41': 'Promising',
    r'51': 'New_Customers',
    r'[4-5][2-3]': 'Potential_Loyalists',
    r'5[4-5]': 'Champions'
}

In [39]:
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.00000,166,11,372.86000,2,5,2,252,25
12347.00000,4,2,1323.32000,5,2,4,524,52
12348.00000,75,1,222.16000,2,1,1,211,21
12349.00000,44,3,2671.14000,3,3,5,335,33
12351.00000,12,1,300.93000,5,1,2,512,51


In [40]:
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.00000,166,11,372.86000,2,5,2,252,Cant_Loose
12347.00000,4,2,1323.32000,5,2,4,524,Potential_Loyalists
12348.00000,75,1,222.16000,2,1,1,211,Hibernating
12349.00000,44,3,2671.14000,3,3,5,335,Need_Attention
12351.00000,12,1,300.93000,5,1,2,512,New_Customers


In [41]:
rfm['Segment'].value_counts()

Segment
Hibernating            1017
Loyal_Customers         742
Champions               663
At_Risk                 611
Potential_Loyalists     517
About_to_Sleep          343
Need_Attention          207
Promising                87
Cant_Loose               77
New_Customers            50
Name: count, dtype: int64